In [1]:
# !pip install -r requirements.txt

In [5]:
import snowflake.snowpark.functions as F
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import StructType, StructField, FloatType
from snowflake.snowpark import Session
import os
import json
import pandas as pd
import numpy as np

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
import mlflow
import mlflow.sklearn
import mlflow.xgboost
import xgboost as xgb

In [6]:
connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "schema": "SENSOR",
    "database": "THINGSBOARD",
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
}

session = Session.builder.configs(connection_parameters).create()

In [12]:
co_df = session.table("CO_TABLE")

In [13]:
co_df.show(5)

------------------------------
|"TS"                 |"CO"  |
------------------------------
|2024-01-02 12:30:00  |5.0   |
|2024-01-02 12:20:00  |2.7   |
|2024-01-02 12:10:00  |5.0   |
|2024-01-02 12:00:00  |2.8   |
|2024-01-02 11:50:00  |5.0   |
------------------------------



In [14]:
from snowflake.snowpark.functions import col, date_part, dayofweek, iff


# Add time features to existing DataFrame 
co_df = co_df.withColumn('day', date_part('day', col('TS'))) \
    .withColumn('month', date_part('month', col('TS'))) \
    .withColumn('hour', date_part('hour', col('TS'))) \
    .withColumn('minute', date_part('minute', col('TS'))) \
    .withColumn('day_of_week', dayofweek(col('TS'))) 

co_df.show(5)

------------------------------------------------------------------------------------
|"TS"                 |"CO"  |"DAY"  |"MONTH"  |"HOUR"  |"MINUTE"  |"DAY_OF_WEEK"  |
------------------------------------------------------------------------------------
|2024-01-02 12:30:00  |5.0   |2      |1        |12      |30        |2              |
|2024-01-02 12:20:00  |2.7   |2      |1        |12      |20        |2              |
|2024-01-02 12:10:00  |5.0   |2      |1        |12      |10        |2              |
|2024-01-02 12:00:00  |2.8   |2      |1        |12      |0         |2              |
|2024-01-02 11:50:00  |5.0   |2      |1        |11      |50        |2              |
------------------------------------------------------------------------------------



In [9]:
df =environmental_df.toPandas()

In [10]:
df.head()

,TS,CO
0,2024-01-02 12:30:00,5.0
1,2024-01-02 12:20:00,2.7
2,2024-01-02 12:10:00,5.0
3,2024-01-02 12:00:00,2.8
4,2024-01-02 11:50:00,5.0


In [11]:
df = df.sort_values('TS').reset_index(drop=True)

In [13]:
# Extract hour and minute
df['hour'] = df['TS'].dt.hour
df['minute'] = df['TS'].dt.minute

# Cyclical encoding
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 60)
df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 60)


In [14]:
df = df.dropna().reset_index(drop=True)

In [15]:
df.head()

,TS,CO,CO_lag_1,CO_lag_2,CO_lag_3,CO_roll_mean_3,CO_roll_std_3,hour,minute,hour_sin,hour_cos,minute_sin,minute_cos
0,2024-01-01 00:30:00,4.4,5.0,2.8,5.0,4.066667,1.137248,0,30,0.000000,1.000000,5.665539e-16,-1.0
1,2024-01-01 00:40:00,5.0,4.4,5.0,2.8,4.800000,0.346410,0,40,0.000000,1.000000,-8.660254e-01,-0.5
2,2024-01-01 00:50:00,4.4,5.0,4.4,5.0,4.600000,0.346410,0,50,0.000000,1.000000,-8.660254e-01,0.5
3,2024-01-01 01:00:00,5.0,4.4,5.0,4.4,4.800000,0.346410,1,0,0.258819,0.965926,0.000000e+00,1.0
4,2024-01-01 01:10:00,2.9,5.0,4.4,5.0,4.100000,1.081665,1,10,0.258819,0.965926,8.660254e-01,0.5


In [16]:
# Define feature columns
feature_cols = [
    'CO_lag_1', 'CO_lag_2', 'CO_lag_3',
    'CO_roll_mean_3', 'CO_roll_std_3',
    'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos'
]

X = df[feature_cols]
y = df['CO']



In [17]:
import mlflow

mlflow.set_tracking_uri("https://legendary-dollop-7vr49x7r7v9pcxwrw-5000.app.github.dev/")


In [18]:
mlflow.set_experiment("Time_Series_CO_Predict")


<Experiment: artifact_location='mlflow-artifacts:/artifacts/4', creation_time=1732377636258, experiment_id='4', last_update_time=1732377636258, lifecycle_stage='active', name='Time_Series_CO_Predict', tags={}>

In [19]:
tscv = TimeSeriesSplit(n_splits=3)

# Start MLflow Run
with mlflow.start_run(run_name="RandomForest_Regressor_Run") as run:
    # Define and Train the Model
    rf = RandomForestRegressor(n_estimators=200, max_depth=None, min_samples_split=2, random_state=42)
    rf.fit(X, y)

    # Predictions (Using the entire dataset for simplicity; consider using a separate test set)
    predictions = rf.predict(X)

    # Calculate Metrics
    mae = mean_absolute_error(y, predictions)
    rmse = np.sqrt(mean_squared_error(y, predictions))

    # Log Parameters
    mlflow.log_param("model", "RandomForestRegressor")
    mlflow.log_param("n_estimators", 200)
    mlflow.log_param("max_depth", None)
    mlflow.log_param("min_samples_split", 2)
    mlflow.log_param("random_state", 42)

    # Log Metrics
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("RMSE", rmse)

    # Log the Model
    mlflow.sklearn.log_model(rf, "model")

    print(f"Run ID: {run.info.run_id}")
    print(f"Logged MAE: {mae:.2f}, RMSE: {rmse:.2f}")

2024/11/23 16:52:59 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2024/11/23 16:53:02 WARNING mlflow.models.model: Model logged without a signature and input example. Ple

Run ID: 0bf0ce7a0c9c488aa418d96546aa52df
Logged MAE: 0.02, RMSE: 0.05
🏃 View run RandomForest_Regressor_Run at: https://legendary-dollop-7vr49x7r7v9pcxwrw-5000.app.github.dev/#/experiments/4/runs/0bf0ce7a0c9c488aa418d96546aa52df
🧪 View experiment at: https://legendary-dollop-7vr49x7r7v9pcxwrw-5000.app.github.dev/#/experiments/4


In [20]:
# Get the Run ID from the previous step
run_id = run.info.run_id

# Register the model
model_name = "Time_Series_CO_Model"
result = mlflow.register_model(
    f"runs:/{run_id}/model",
    model_name
)

print(f"Registered Model: {result.name}, Version: {result.version}")


Registered model 'Time_Series_CO_Model' already exists. Creating a new version of this model...
2024/11/23 16:53:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Time_Series_CO_Model, version 4


Registered Model: Time_Series_CO_Model, Version: 4


Created version '4' of model 'Time_Series_CO_Model'.


In [21]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Transition the model to 'Staging'
client.transition_model_version_stage(
    name=model_name,
    version=result.version,
    stage="Staging",
    archive_existing_versions=True
)

print(f"Model {model_name} version {result.version} transitioned to Staging.")


Model Time_Series_CO_Model version 4 transitioned to Staging.


/tmp/ipykernel_66/1892959298.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [22]:
df_new

NameError: name 'df_new' is not defined

In [ ]:
import pandas as pd
import numpy as np

import mlflow
import mlflow.sklearn
mlflow.set_tracking_uri("https://legendary-dollop-7vr49x7r7v9pcxwrw-5000.app.github.dev/")

model_name = "Time_Series_CO_Model"
model_stage = "Staging"

# Load the model from the registry
model = mlflow.sklearn.load_model(
    model_uri=f"models:/{model_name}/{model_stage}"
)

print("Model loaded successfully from MLflow Model Registry.")
ts = [1672531200000, 1672531260000, 1672531320000, 1672531380000, 1672531440000]

# Sample data for CO
co = [10.5, 12.3, 9.8, 11.7, 10]

# Create the DataFrame
df_new = pd.DataFrame({'TS': ts, 'CO': co})
df_new['TS'] = pd.to_datetime(df_new['TS'], unit='ms')

# Sort by timestamp
df_new = df_new.sort_values('TS').reset_index(drop=True)
for lag in range(1, 4):
    df_new[f'CO_lag_{lag}'] = df_new['CO'].shift(lag)

# Generating rolling statistics
df_new['CO_roll_mean_3'] = df_new['CO'].rolling(window=3).mean()
df_new['CO_roll_std_3'] = df_new['CO'].rolling(window=3).std()

# Extracting time-based features
df_new['hour'] = df_new['TS'].dt.hour
df_new['minute'] = df_new['TS'].dt.minute

# Cyclical Encoding
df_new['hour_sin'] = np.sin(2 * np.pi * df_new['hour'] / 24)
df_new['hour_cos'] = np.cos(2 * np.pi * df_new['hour'] / 24)
df_new['minute_sin'] = np.sin(2 * np.pi * df_new['minute'] / 60)
df_new['minute_cos'] = np.cos(2 * np.pi * df_new['minute'] / 60)

# Drop rows with NaN values resulting from lagging and rolling
df_new= df_new.dropna().reset_index(drop=True)
feature_cols = [
            'CO_lag_1', 'CO_lag_2', 'CO_lag_3',
            'CO_roll_mean_3', 'CO_roll_std_3',
            'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos'
        ]

X_input = df_new[feature_cols]
predicted_CO = model.predict(X_input)


In [ ]:
predicted_CO

In [ ]:
latest_row

In [ ]:
X

In [ ]:
predicted_CO = model.predict(X_input)

In [ ]:
predicted_CO

In [ ]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
mlflow.set_tracking_uri("https://legendary-dollop-7vr49x7r7v9pcxwrw-5000.app.github.dev/")
model_name = "Time_Series_CO_Model"
model_version = 1  

model = mlflow.sklearn.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)
ts = [1672531200000, 1672531260000, 1672531320000, 1672531380000, 1672531440000]


co = [10.5, 12.3, 9.8, 11.7, 10]
df_new = pd.DataFrame({'TS': ts, 'CO': co})
df_new['TS'] = pd.to_datetime(df_new['TS'], unit='ms')

# Sort by timestamp
df_new = df_new.sort_values('TS').reset_index(drop=True)

# Create lag features
for lag in range(1, 4):
    df_new[f'CO_lag_{lag}'] = df_new['CO'].shift(lag)

# Generate rolling statistics
df_new['CO_roll_mean_3'] = df_new['CO'].rolling(window=3).mean()
df_new['CO_roll_std_3'] = df_new['CO'].rolling(window=3).std()

# Extract time-based features
df_new['hour'] = df_new['TS'].dt.hour
df_new['minute'] = df_new['TS'].dt.minute

# Cyclical Encoding
df_new['hour_sin'] = np.sin(2 * np.pi * df_new['hour'] / 24)
df_new['hour_cos'] = np.cos(2 * np.pi * df_new['hour'] / 24)
df_new['minute_sin'] = np.sin(2 * np.pi * df_new['minute'] / 60)
df_new['minute_cos'] = np.cos(2 * np.pi * df_new['minute'] / 60)

# Drop rows with NaN values resulting from lagging and rolling
df_new = df_new.dropna().reset_index(drop=True)

# Define feature columns
feature_cols = [
    'CO_lag_1', 'CO_lag_2', 'CO_lag_3',
    'CO_roll_mean_3', 'CO_roll_std_3',
    'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos'
]

# Prepare input features
X_input = df_new[feature_cols]

# Make predictions
predicted_CO = model.predict(X_input)

print("Predictions:", predicted_CO)


In [ ]:
##  ds is timestamp
##
import pandas as pd
import numpy as np
from prophet import Prophet

import pandas as pd
import numpy as np

import mlflow
import mlflow.sklearn

# Set the MLflow tracking URI
mlflow.set_tracking_uri("https://legendary-dollop-7vr49x7r7v9pcxwrw-5000.app.github.dev/")

model_name = "Time_Series_CO_Model"
model_version = 1  # Specify the exact version you want to load

# Load the model by specifying its version
model = mlflow.sklearn.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

print("Model loaded successfully from MLflow Model Registry.")



df = pd.DataFrame()
df['ds'] = pd.to_datetime(input_x, unit='ms')
df['hour'] = df['ds'].dt.hour
df['minute'] = df['ds'].dt.minute
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 60)
df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 60)


df['CO'] = np.array(input_y)
for lag in range(1, 4):
    df_new[f'CO_lag_{lag}'] = df_new['CO'].shift(lag)

# Generate rolling statistics
df['CO_roll_mean_3'] = df['CO'].rolling(window=3).mean()
df['CO_roll_std_3'] = df['CO'].rolling(window=3).std()

df = df.dropna().reset_index(drop=True)

feature_cols = [
    'CO_lag_1', 'CO_lag_2', 'CO_lag_3',
    'CO_roll_mean_3', 'CO_roll_std_3',
    'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos'
]

X_input = df[feature_cols]
model.predict(X_input)
forecast = model.predict(future)
output_y = forecast['yhat'].tolist()
print(f"result: {output_y}")
return output_y